In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# NYC Food Access — Neighborhood-Level (NTA2020) Analysis

This notebook focuses on a **single standardized neighborhood unit**: NYC Neighborhood
Tabulation Areas (NTA2020). We assume the **metrics CSV (with `nta2020` codes)** is the
source of truth for:
- Which neighborhoods exist
- Their IDs
- Metrics like supply gap, food insecurity, unemployment, etc.

In [43]:
CENSUS_TRACTS_GEOJSON = "raw/census_tracts.geojson"   # e.g., 2020 Census tracts GeoJSON/GeoPackage/SHAPE
PANTRIES_PATH      = "data/tabular/pantries.csv"    # can be a GeoJSON of points OR a CSV with lat/lon columns
PANTRIES_GEO = "data/geo/pantries.geojson"
SUPPLY_GAP_CSV     = "raw/Emergency_Food_Supply_Gap_20251110.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None
POPULATION = "data/tabular/nyc_population.csv"

In [24]:

import pandas as pd
import geopandas as gpd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

# If PANTRIES_PATH is CSV, specify columns:
PANTRY_LAT_COL = "latitude"
PANTRY_LON_COL = "longitude"

CRS_LATLON    = "EPSG:4326"
CRS_PROJECTED = "EPSG:6539"  # NYC-appropriate projected CRS


In [84]:
population = pd.read_csv(POPULATION)   
tracts = gpd.read_file(CENSUS_TRACTS_GEOJSON)  # or your cleaned tract GeoJSON
# make sure it has at least ['geoid','nta2020','ntaname']
pantries = pd.read_csv(PANTRIES_PATH)
metrics = pd.read_csv(SUPPLY_GAP_CSV)

# 3️⃣ Turn pantries into a GeoDataFrame
gpantries = gpd.GeoDataFrame(
    pantries,
    geometry=[Point(xy) for xy in zip(pantries["lng"], pantries["lat"])],
    crs="EPSG:4326"   # WGS84 coordinates
)

# 4️⃣ Ensure coordinate systems match
tracts = tracts.to_crs(epsg=4326)
tracts.shape

(2325, 18)

In [86]:
population.rename(columns={"CensusTract": "geoid"}, inplace=True)
population['geoid'] = population['geoid'].astype(int)
tracts['geoid'] = tracts['geoid'].astype(int)

In [88]:
tracts = tracts.merge(population, on="geoid", how="left")

In [ ]:
tracts.to_csv("data/model/tracts_table.csv")

## 1. Aggregate NTA tracts

In [96]:
# Rename for consistency
metrics = metrics.rename(columns={
    "Neighborhood Tabulation Area NTA)": "nta2020",
    "Neighborhood Tabulation Area (NTA) Name": "ntaname"
})

In [97]:
metrics.head()

,Year,nta2020,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank
0,2025-01-01,BK0101,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0
1,2025-01-01,BK0102,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0
2,2025-01-01,BK0103,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0
3,2025-01-01,BK0104,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0
4,2025-01-01,BK0201,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0


In [98]:
metrics["nta2020"] = metrics["nta2020"].str.strip().str.upper()
metrics["Year"] = pd.to_datetime(metrics["Year"], format='%Y')

idx = metrics.groupby("nta2020")["Year"].idxmax()
metrics = metrics.loc[idx].reset_index(drop=True)

In [99]:
# 1️⃣ Keep only the NTA geometries from tracts (since each tract already has nta2020)
nta_geoms = (tracts[['nta2020', 'geoid','geometry']]
             .dissolve(by='nta2020')        # merge all tracts for same nta2020
             .reset_index())

print(f"Unique NTAs (from tracts): {nta_geoms.shape[0]}")

Unique NTAs (from tracts): 262


## Merge metrics and nta geometry gap

In [100]:
# 2️⃣ Join metrics onto those polygons
nta_enriched = nta_geoms.merge(metrics, on='nta2020', how='left')
print(f"NTA polygons after merge: {nta_enriched.shape}")

nta_enriched["Supply Gap (lbs.)"].isna().sum()

NTA polygons after merge: (262, 11)


65

In [101]:
nta_enriched.head()

,nta2020,geometry,geoid,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank
0,BK0101,"POLYGON ((-73.95165 40.72349, -73.9525 40.7232...",36047056301,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0
1,BK0102,"POLYGON ((-73.96485 40.70745, -73.96593 40.707...",36047051300,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0
2,BK0103,"POLYGON ((-73.95137 40.69963, -73.95154 40.699...",36047050900,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0
3,BK0104,"POLYGON ((-73.94042 40.70108, -73.94193 40.700...",36047048900,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0
4,BK0201,"POLYGON ((-73.99045 40.69372, -73.99059 40.693...",36047000301,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0


In [102]:
nta_enriched.dropna(subset=['Supply Gap (lbs.)'], inplace=True)

## Pantries per tract and neighborhood

In [103]:
# 3️⃣ Spatially assign pantries to NTA polygons
pantries_in_nta = gpd.sjoin(gpantries, nta_enriched, how='left', predicate='within')
pantry_counts = (pantries_in_nta
                 .groupby('nta2020')
                 .size()
                 .rename('pantry_count')
                 .reset_index())

In [104]:
nta_enriched = nta_enriched.merge(pantry_counts, on='nta2020', how='left')
nta_enriched['pantry_count'] = nta_enriched['pantry_count'].fillna(0).astype(int)

In [105]:
# Export
nta_enriched.drop(columns=['geometry']).to_csv('data/tabular/nta_with_metrics.csv', index=False)
nta_enriched.to_file('data/geo/nta_with_metrics.geojson', driver='GeoJSON')

In [106]:
# Centroids per neighborhood
nta_proj = nta_enriched.to_crs(CRS_PROJECTED)

# 2. Compute centroids in projected CRS
nta_proj["centroid_geom"] = nta_proj.geometry.centroid

# 3. Convert centroids back to WGS84 lat/lon for mapping/export
nta_centroids_ll = nta_proj.set_geometry("centroid_geom").to_crs(CRS_LATLON)

# 4. Extract simple coordinates
nta_centroids_ll["centroid_lon"] = nta_centroids_ll.geometry.x
nta_centroids_ll["centroid_lat"] = nta_centroids_ll.geometry.y

nta_centroids_ll.head()

,nta2020,geometry,geoid,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank,pantry_count,centroid_geom,centroid_lon,centroid_lat
0,BK0101,"POLYGON ((997652.824 202865.648, 997417.375 20...",36047056301,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0,1,POINT (-73.94952 40.7295),-73.949516,40.729500
1,BK0102,"POLYGON ((993995.781 197022.868, 993695.322 19...",36047051300,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0,1,POINT (-73.95862 40.71491),-73.958621,40.714914
2,BK0103,"POLYGON ((997733.467 194175.636, 997687.83 194...",36047050900,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0,1,POINT (-73.95666 40.70324),-73.956659,40.703242
3,BK0104,"POLYGON ((1000769.963 194703.194, 1000351.303 ...",36047048900,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0,1,POINT (-73.93649 40.7133),-73.936488,40.713299
4,BK0201,"POLYGON ((986897.487 192017.706, 986860.041 19...",36047000301,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0,0,POINT (-73.99483 40.69547),-73.994830,40.695465


In [107]:
nta_proj = nta_enriched.to_crs(CRS_PROJECTED)
centroids = nta_proj.copy()
centroids['geometry'] = nta_proj.centroid

centroids.head()

,nta2020,geometry,geoid,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank,pantry_count
0,BK0101,POINT (998241.957 205056.938),36047056301,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0,1
1,BK0102,POINT (995721.016 199741.715),36047051300,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0,1
2,BK0103,POINT (996267.118 195489.486),36047050900,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0,1
3,BK0104,POINT (1001857.171 199157.092),36047048900,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0,1
4,BK0201,POINT (985683.594 192653.281),36047000301,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0,0


In [ ]:
centroids.to_csv("data/model/nta_table.csv", index=False)

### New Pantries Points

In [109]:
# ---- Load pantry locations ----
pantry_path = Path(PANTRIES_GEO)
assert pantry_path.exists(), f"Pantry file not found: {pantry_path}"


pantries = gpd.read_file(pantry_path)
if pantries.crs is None:
    print("⚠️ Pantries have no CRS; assuming WGS84")
    pantries = pantries.set_crs(CRS_LATLON)

pantries_ll = pantries.to_crs(CRS_LATLON)
pantries_proj = pantries_ll.to_crs(CRS_PROJECTED)

print("Pantries:", pantries_ll.shape)
pantries_ll.head(2)

Pantries: (515, 125)


,FID,type_fp,type_sk,program,org_phone,distadd,distboro,distzip,dist_location_info,fp_days_orig,fp_hours_orig,sk_days_orig,sk_hours_orig,fp_mon,fp_mon_open1,fp_mon_close1,fp_mon_open2,fp_mon_close2,fp_tue,fp_tue_open1,fp_tue_close1,fp_tue_open2,fp_tue_close2,fp_wed,fp_wed_open1,fp_wed_close1,fp_wed_open2,fp_wed_close2,fp_thu,fp_thu_open1,...,program_type,fp_mon_open3,fp_mon_close3,fp_tue_open3,fp_tue_close3,fp_wed_open3,fp_wed_close3,fp_thu_open3,fp_thu_close3,fp_fri_open3,fp_fri_close3,fp_sat_open3,fp_sat_close3,fp_sun_open3,fp_sun_close3,sk_mon_open3,sk_mon_close3,sk_tue_open3,sk_tue_close3,sk_wed_open3,sk_wed_close3,sk_thu_open3,sk_thu_close3,sk_fri_open3,sk_fri_close3,sk_sat_open3,sk_sat_close3,sk_sun_open3,sk_sun_close3,geometry
0,1,FP,None,CHURCH OF ST. NICHOLAS OF TOLENTINE,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,TUE,9-11AM,None,None,closed,None,None,None,None,open,09:00 AM,11:00 AM,None,None,closed,None,None,None,None,closed,None,...,FP,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90567 40.86222)
1,2,FP,,BREAD OF LIFE FOOD PANTRY,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,SAT,1:30 - 4:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,FP,,,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.87854 40.82642)


In [110]:
tracts_for_join = tracts[['geoid', "nta2020", "geometry"]].set_geometry("geometry")

pantries_with_geo_ids = gpd.sjoin(
    pantries_ll,
    tracts_for_join,
    how="left",
    predicate="within"
)

# Clean up the sjoin artifact
pantries_with_geo_ids = pantries_with_geo_ids.drop(columns=["index_right"], errors="ignore")

pantries_with_geo_ids.head()

,FID,type_fp,type_sk,program,org_phone,distadd,distboro,distzip,dist_location_info,fp_days_orig,fp_hours_orig,sk_days_orig,sk_hours_orig,fp_mon,fp_mon_open1,fp_mon_close1,fp_mon_open2,fp_mon_close2,fp_tue,fp_tue_open1,fp_tue_close1,fp_tue_open2,fp_tue_close2,fp_wed,fp_wed_open1,fp_wed_close1,fp_wed_open2,fp_wed_close2,fp_thu,fp_thu_open1,...,fp_mon_close3,fp_tue_open3,fp_tue_close3,fp_wed_open3,fp_wed_close3,fp_thu_open3,fp_thu_close3,fp_fri_open3,fp_fri_close3,fp_sat_open3,fp_sat_close3,fp_sun_open3,fp_sun_close3,sk_mon_open3,sk_mon_close3,sk_tue_open3,sk_tue_close3,sk_wed_open3,sk_wed_close3,sk_thu_open3,sk_thu_close3,sk_fri_open3,sk_fri_close3,sk_sat_open3,sk_sat_close3,sk_sun_open3,sk_sun_close3,geometry,geoid,nta2020
0,1,FP,None,CHURCH OF ST. NICHOLAS OF TOLENTINE,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,TUE,9-11AM,None,None,closed,None,None,None,None,open,09:00 AM,11:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90567 40.86222),36005025500,BX0701
1,2,FP,,BREAD OF LIFE FOOD PANTRY,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,SAT,1:30 - 4:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.87854 40.82642),36005005002,BX0901
2,4,FP,,CHANCE FOR CHILDREN,(347) 616-3228,"11 Mc Keever Pl, Brooklyn, New York, 11225",BK,11225,1ST FLOOR (BASEMENT LEVEL),FRI,10AM-12:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.95848 40.66569),36047032500,BK0901
3,6,FP,None,HEALTH ESSENTIAL ASSOCIATION INC (BK),(646) 515-6898,"2101 E 16th St, Brooklyn, New York, 11229",BK,11229,2ND FLOOR,FRI (4TH),10AM-12PM,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.95528 40.59861),36047058000,BK1502
4,10,FPHA,None,AIDS CENTER OF QUEENS COUNTY (WOODSIDE),(718) 472-9400,"62-07 Woodside Ave, Woodside, New York, 11377",QN,11377,3RD FLOOR,"TUE, THUR",9-11AM,None,None,closed,None,None,None,None,open,08:00 AM,10:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90219 40.74466),36081026100,QN0203


In [111]:
pantries_with_geo_ids['id'] = pantries_with_geo_ids.index

In [112]:
pantries_with_geo_ids

,FID,type_fp,type_sk,program,org_phone,distadd,distboro,distzip,dist_location_info,fp_days_orig,fp_hours_orig,sk_days_orig,sk_hours_orig,fp_mon,fp_mon_open1,fp_mon_close1,fp_mon_open2,fp_mon_close2,fp_tue,fp_tue_open1,fp_tue_close1,fp_tue_open2,fp_tue_close2,fp_wed,fp_wed_open1,fp_wed_close1,fp_wed_open2,fp_wed_close2,fp_thu,fp_thu_open1,...,fp_tue_open3,fp_tue_close3,fp_wed_open3,fp_wed_close3,fp_thu_open3,fp_thu_close3,fp_fri_open3,fp_fri_close3,fp_sat_open3,fp_sat_close3,fp_sun_open3,fp_sun_close3,sk_mon_open3,sk_mon_close3,sk_tue_open3,sk_tue_close3,sk_wed_open3,sk_wed_close3,sk_thu_open3,sk_thu_close3,sk_fri_open3,sk_fri_close3,sk_sat_open3,sk_sat_close3,sk_sun_open3,sk_sun_close3,geometry,geoid,nta2020,id
0,1,FP,None,CHURCH OF ST. NICHOLAS OF TOLENTINE,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,TUE,9-11AM,None,None,closed,None,None,None,None,open,09:00 AM,11:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90567 40.86222),36005025500,BX0701,0
1,2,FP,,BREAD OF LIFE FOOD PANTRY,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,SAT,1:30 - 4:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.87854 40.82642),36005005002,BX0901,1
2,4,FP,,CHANCE FOR CHILDREN,(347) 616-3228,"11 Mc Keever Pl, Brooklyn, New York, 11225",BK,11225,1ST FLOOR (BASEMENT LEVEL),FRI,10AM-12:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.95848 40.66569),36047032500,BK0901,2
3,6,FP,None,HEALTH ESSENTIAL ASSOCIATION INC (BK),(646) 515-6898,"2101 E 16th St, Brooklyn, New York, 11229",BK,11229,2ND FLOOR,FRI (4TH),10AM-12PM,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.95528 40.59861),36047058000,BK1502,3
4,10,FPHA,None,AIDS CENTER OF QUEENS COUNTY (WOODSIDE),(718) 472-9400,"62-07 Woodside Ave, Woodside, New York, 11377",QN,11377,3RD FLOOR,"TUE, THUR",9-11AM,None,None,closed,None,None,None,None,open,08:00 AM,10:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90219 40.74466),36081026100,QN0203,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,815,FP,None,FIRST UNITED CHRISTIAN CHURCH (FP),None,"109 Victory Blvd, Staten Island, NY, 10301, USA",SI,10301,None,None,None,None,None,closed,None,None,None,None,open,10:00 AM,01:00 PM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-74.08022 40.63766),36085001100,SI0101,510
511,816,FP,None,MAS STATEN ISLAND CENTER FOOD PANTRY (FP),(800) 668-0742,"180 Burgher Ave, Staten Island, NY, 10304, USA",SI,10304,None,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-74.09192 40.59242),36085009602,SI0201,511
512,817,FP,None,MASJID AR RAHMAN FOOD PANTRY,(718) 740-5025,"98-10 211th St, Queens Village, NY, 11429, USA",QN,11429,None,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [ ]:
pantries_with_geo_ids.to_csv("data/model/pantries_table.csv", index=False)